In [2]:
!pip install fastembed


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!pip install fastembed numpy qdrant-client requests


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import numpy as np
from fastembed import TextEmbedding

embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

query = 'I just discovered the course. Can I join now?'
query_embedding_list = list(embedding_model.embed([query]))
query_embedding = query_embedding_list[0]

print(f"vetcor: {query_embedding.shape}")
min_value = np.min(query_embedding)
print(f"min_value: {min_value}")

Fetching 5 files: 100%|██████████| 5/5 [00:14<00:00,  2.80s/it]


vetcor: (512,)
min_value: -0.11726373885183883


In [6]:
import numpy as np

np.linalg.norm(query_embedding)

np.float64(1.0)

In [8]:
doc = 'Can I still join the course after the start date?'
doc_embedding_list = list(embedding_model.embed([doc]))
doc_embedding = doc_embedding_list[0]

cosine_similarity = query_embedding.dot(doc_embedding)

print(f" the cosine similarity: {cosine_similarity}")

 the cosine similarity: 0.9008528895674548


In [9]:
documents = [
    {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'},
    {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'},
    {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'},
    {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.', 'section': 'General course-related questions', 'question': 'Course - What can I do before the course starts?', 'course': 'data-engineering-zoomcamp'},
    {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.', 'section': 'General course-related questions', 'question': 'How can we contribute to the course?', 'course': 'data-engineering-zoomcamp'}
]

texts = [d['text'] for d in documents]
text_embeddings_list = list(embedding_model.embed(texts))
V = np.array(text_embeddings_list)

scores = V.dot(query_embedding)
highest_score_index = np.argmax(scores)

print(f"scores: {scores}")
print(f"highest_score_index: {highest_score_index}")

scores: [0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]
highest_score_index: 1


In [12]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

full_text_embeddings_list = list(embedding_model.embed(full_texts))
V_full = np.array(full_text_embeddings_list)

new_scores = V_full.dot(query_embedding)
highest_new_score_index = np.argmax(new_scores)

print(f"new_scores: {new_scores}")
print(f"highest_score_index: {highest_score_index}")

new_scores: [0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]
highest_score_index: 1


In [13]:
from fastembed.embedding import FlagEmbedding

model_list = FlagEmbedding.list_supported_models()

min_dim = float('inf')
for model_info in model_list:
    if 'dim' in model_info and model_info['dim'] < min_dim:
        min_dim = model_info['dim']

print(f"min_dim: {min_dim}")

2025-08-24 05:35:01.950 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


min_dim: 384


In [14]:
import requests
from qdrant_client import QdrantClient, models

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents_ml = []
for course in documents_raw:
    if course['course'] == 'machine-learning-zoomcamp':
        for doc in course['documents']:
            doc['course'] = course['course']
            documents_ml.append(doc)



embedding_model_small = TextEmbedding(model_name="BAAI/bge-small-en")

client = QdrantClient(":memory:")

collection_name = "ml-zoomcamp-faq"

vector_size = embedding_model_small.get_text_embedding_dimension(model_name="BAAI/bge-small-en")

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=vector_size,
        distance=models.Distance.COSINE
    )
)

texts_to_embed = [doc['question'] + ' ' + doc['text'] for doc in documents_ml]
embeddings = embedding_model_small.embed(texts_to_embed)

client.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=embedding.tolist(), 
            payload=doc
        )
        for idx, (embedding, doc) in enumerate(zip(embeddings, documents_ml))
    ],
    wait=True
)

query = 'I just discovered the course. Can I join now?'
query_vector_small = embedding_model_small.embed([query])[0]

search_results = client.search(
    collection_name=collection_name,
    query_vector=query_vector_small,
    limit=1 
)

highest_score = search_results[0].score
print(f"\nhighest_score: {highest_score}")

Fetching 5 files: 100%|██████████| 5/5 [00:06<00:00,  1.20s/it]


AttributeError: 'TextEmbedding' object has no attribute 'get_text_embedding_dimension'